In [195]:
import math
household_income_2013_acs5 = [
            #dict(min=math.nan, max=9999, n=6, moe=1),
            #dict(min=10000, max=14999, n=1, moe=1),
            #dict(min=15000, max=19999, n=8, moe=1),
            #dict(min=20000, max=24999, n=7, moe=1),
            #dict(min=25000, max=29999, n=2, moe=1),
            dict(min=30000, max=34999, n=900, moe=8),
            #dict(min=35000, max=39999, n=7, moe=1),
            #dict(min=40000, max=44999, n=4, moe=1),
            #dict(min=45000, max=49999, n=8, moe=1),
            #dict(min=50000, max=59999, n=6, moe=1),
            #dict(min=60000, max=74999, n=7, moe=1),
            #dict(min=75000, max=99999, n=2, moe=0.25),
            #dict(min=100000, max=124999, n=7, moe=1),
            #dict(min=125000, max=149999, n=10, moe=1),
            #dict(min=150000, max=199999, n=8, moe=1),
            #dict(min=200000, max=math.nan, n=18, moe=10)
        ]

household_income_la_2013_acs1 = [
      #dict(min=0, max=9999, n=1382, moe = 50),
      #dict(min=10000, max=14999, n=2377, moe = 50),
      #dict(min=15000, max=19999, n=1332, moe = 50),
      #dict(min=20000, max=24999, n=3129, moe = 50),
      dict(min=25000, max=29999, n=1927, moe = 50),
      #dict(min=30000, max=34999, n=1825, moe = 50),
      #dict(min=35000, max=39999, n=1567, moe = 50),
      #dict(min=40000, max=44999, n=1996, moe = 50),
      #dict(min=45000, max=49999, n=1757, moe = 50),
      #dict(min=50000, max=59999, n=3523, moe = 50),
      #dict(min=60000, max=74999, n=4360, moe = 50),
      #dict(min=75000, max=99999, n=6424, moe = 50),
      #dict(min=100000, max=124999, n=5257, moe = 50),
      #dict(min=125000, max=149999, n=3485, moe = 50),
      #dict(min=150000, max=199999, n=2926, moe = 50),
      #dict(min=200000, max=500000, n=4215, moe = 50)
  ]


test_input = [household_income_2013_acs5,household_income_la_2013_acs1]

makeup_1 = [0.8, 0.2]
makeup_2 =  [0.3, 0.7]

makeup_input = [makeup_1, makeup_2]


In [8]:
def disaggregate_sum(data_list, makeup, deterministic = True, simulations=50):
    
    #  if you just want an estimate and ignore margin of error
    if deterministic:
        results_sum = []
        for i in range(len(data_list)):
            total = []
            results_sum.append([data_list[i][0]['n']*z for z in makeup[i]]) #  get proportion of n going to each new geography
        rr = numpy.vstack(results_sum) #  get into array format
        est = numpy.apply_along_axis(sum, 0, rr) #  sum each column (total for each new geography)
        return est, None
    # otherwise deal with margin of error via simulation
    else:
        simulation_results = []
        for s in range(simulations):
            results_sum = []
            for i in range(len(data_list)):
                total = []
                se = data_list[i][0]['moe'] / 1.645 #  convert moe to se
                new_n = round(numpy.random.normal(data_list[i][0]['n'], se)) # u se moe to introduce randomness into number in bin
                new_n = int(new_n) #  clean it up
                results_sum.append([new_n*z for z in makeup[i]]) #  get proportion of simulated n going to each new geography
            rr = numpy.vstack(results_sum) # get into array format
            simulation_results.append(numpy.apply_along_axis(sum, 0, rr)) #  sum each column (total for each new geography)
        ss = numpy.vstack(simulation_results) #  get simulation into array format
        est = numpy.apply_along_axis(numpy.mean, 0, ss) #  mean across simulations
        t1 = numpy.apply_along_axis(numpy.quantile, axis=0, arr =ss, q=0.95) - est #  higher quantile across simulations
        t2 = est - numpy.apply_along_axis(numpy.quantile, axis=0, arr = ss, q=0.05) #  lower quantile across simulations
        margin_of_error = numpy.amax(numpy.column_stack((t1,t2)), 1) #  take the larger of each row to be conservative
        return est, margin_of_error

In [12]:
disaggregate_sum(test_input, makeup_input, deterministic = True, simulations=50)

(array([1247.1, 1409.9]), None)

In [13]:
disaggregate_sum(test_input, makeup_input, deterministic = False, simulations=50)

(array([1250.174, 1417.406]), array([18.619, 39.889]))

In [27]:
def disaggregate_mean(data_list, makeup, deterministic = True, simulations=50):
    #  if you just want an estimate and ignore margin of error
    if deterministic:
        denom = disaggregate_sum(data_list, makeup, deterministic = True, simulations=50)[0] #  get the weighted sum for denominator of weighted mean
        results = []
        grand_total = []
        for i in range(len(data_list)):
            total = data_list[i][0]['n'] * data_list[i][0]['val'] #  weighted value
            results.append([total*z for z in makeup[i]])  #  partition by new geograph
        rr = numpy.vstack(results) #  get in array format
        rr2 = numpy.apply_along_axis(sum, 0, rr) #  weighted sum
        est = numpy.divide(rr2, denom) #  get the weighted average
        return est, None
    #  otherwise deal with margin of error via simulation
    else:
        #  have to redo simulation can't just use sum call because randomness won't match up
        simulation_results = [] #  weighted sum
        simulation_results_v = [] #  weighted sum and values
        for s in range(simulations):
            results = [] #   intermediate weighted sum
            results_v = [] #  intermediate weighted sum and values
            for i in range(len(data_list)):
                se = data_list[i][0]['moe'] / 1.645 #  convert moe to se
                new_n = round(numpy.random.normal(data_list[i][0]['n'], se))
                #  use moe to introduce randomness into number in bin
                new_n = int(new_n) #  clean it up
                results.append([new_n * z for z in makeup[i]]) 
                results_v.append([new_n * data_list[i][0]['val']*z  for z in makeup[i]])
            rr = numpy.vstack(results) #  get results into array format
            rr_v = numpy.vstack(results_v) #  get results into array format
            simulation_results.append(numpy.apply_along_axis(sum, 0, rr)) #  do sum for numerator of weighted mean
            simulation_results_v.append(numpy.apply_along_axis(sum, 0, rr_v)) #  do sum for denominator of weighted mean
        ss = numpy.vstack(simulation_results) #  get simulation into array format
        ss_v = numpy.vstack(simulation_results_v) #  get simulation into array format
        ssd = numpy.divide(ss_v, ss) #  get weighted mean
        est = numpy.apply_along_axis(numpy.mean, 0, ssd) #  take mean of weighted means across simulations
        t1 = numpy.apply_along_axis(numpy.quantile, axis=0, arr =ssd, q=0.95) - est #  upper quantile of weighted means across simulations
        t2 = est - numpy.apply_along_axis(numpy.quantile, axis=0, arr = ssd, q=0.05) #  lower quantile of weighted means across simulations
        margin_of_error = numpy.amax(numpy.column_stack((t1,t2)), 1) #  take max across rows to be conservative
        return est, margin_of_error


In [5]:
household_income_2013_acs5 = [
            #dict(min=math.nan, val=9999, n=6, moe=1),
            #dict(min=10000, val=14999, n=1, moe=1),
            #dict(min=15000, val=19999, n=8, moe=1),
            #dict(min=20000, val=24999, n=7, moe=1),
            #dict(min=25000, val=29999, n=2, moe=1),
            dict(min=30000, val=34999, n=900, moe=8)#,
            #dict(min=35000, val=39999, n=7, moe=1),
            #dict(min=40000, val=44999, n=4, moe=1),
            #dict(min=45000, val=49999, n=8, moe=1),
            #dict(min=50000, val=59999, n=6, moe=1),
            #dict(min=60000, val=74999, n=7, moe=1),
            #dict(min=75000, val=99999, n=2, moe=0.25),
            #dict(min=100000, val=124999, n=7, moe=1),
            #dict(min=125000, val=149999, n=10, moe=1),
            #dict(min=150000, val=199999, n=8, moe=1),
            #dict(min=200000, val=250000, n=18, moe=10)
        ]

household_income_la_2013_acs1 = [
      #dict(min=0, val=9979, n=1382, moe = 50),
      #dict(min=10000, val=14799, n=2377, moe = 50),
      #dict(min=15000, val=13999, n=1332, moe = 50),
      #dict(min=20000, val=24899, n=3129, moe = 50),
      #dict(min=25000, val=29999, n=1927, moe = 50),
      #dict(min=30000, val=34499, n=1825, moe = 50),
      #dict(min=35000, val=31999, n=1567, moe = 50),
      #dict(min=40000, val=44799, n=1996, moe = 50),
      dict(min=45000, val=49929, n=1757, moe = 50)#,
      #dict(min=50000, val=54999, n=3523, moe = 50),
      #dict(min=60000, val=76999, n=4360, moe = 50),
      #dict(min=75000, val=99939, n=6424, moe = 50),
      #dict(min=100000, val=124499, n=5257, moe = 50),
      #dict(min=125000, val=159999, n=3485, moe = 50),
      #dict(min=150000, val=199099, n=2926, moe = 50),
      #dict(min=200000, val=500000, n=4215, moe = 50)
  ]


test_input = [household_income_2013_acs5,household_income_la_2013_acs1]

makeup_1 = [0.8, 0.2]
makeup_2 =  [0.3, 0.7]

makeup_input = [makeup_1, makeup_2]

In [26]:
import numpy
disaggregate_mean(test_input, makeup_input, deterministic = True, simulations=5)

(array([41309.32234785, 48022.90736932]), None)

In [28]:
disaggregate_mean(test_input, makeup_input, deterministic = False, simulations=5)

[[ 722.4  180.6]
 [ 522.  1218. ]]
[[25283277.6         6320819.4       ]
 [26062938.         60813521.99999999]]
[[ 711.2  177.8]
 [ 518.4 1209.6]]
[[24891288.8  6222822.2]
 [25883193.6 60394118.4]]
[[ 725.6  181.4]
 [ 523.2 1220.8]]
[[25395274.4  6348818.6]
 [26122852.8 60953323.2]]
[[ 720.8  180.2]
 [ 527.1 1229.9]]
[[25227279.2         6306819.8       ]
 [26317575.9        61407677.09999999]]
[[ 714.4  178.6]
 [ 531.3 1239.7]]
[[25003285.6  6250821.4]
 [26527277.7 61896981.3]]


(array([41296.29122872, 48016.85994523]), array([58.16597458, 26.49906107]))